In [1]:
from hugging_face.get_transformer_model_path import get_transformer_model_path
from hugging_face.finetuning_of_the_pretrained_model.preprocessing import preprocess_function
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, TrainingArguments, Trainer, DefaultDataCollator

tokenizer = AutoTokenizer.from_pretrained(get_transformer_model_path())

# import torch
# from transformers import AdamW, AutoTokenizer, AutoModelForSequenceClassification
# # Same as before
checkpoint = get_transformer_model_path()
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForQuestionAnswering.from_pretrained(checkpoint)
# sequences = [    "I've been waiting for a HuggingFace course my whole life.",    "This course is amazing!",]
# batch = tokenizer(sequences, padding=True, truncation=True, return_tensors="pt")
# # This is new
# batch["labels"] = torch.tensor([1, 1])
# optimizer = AdamW(model.parameters())
# loss = model(**batch).loss
# loss.backward()
# optimizer.step()


data_collator = DefaultDataCollator()
dataset = load_dataset(
    path='./dataset'
    )

dataset["train"][0]

tokenized_dataset = dataset.map(preprocess_function, batched=True, remove_columns=dataset["train"].column_names)

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01, )
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator, )
trainer.train()


Using custom data configuration dataset-8702b8da36782627


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset json downloaded and prepared to C:\Users\Knupf\.cache\huggingface\datasets\json\dataset-8702b8da36782627\0.0.0\da492aad5680612e4028e7f6ddc04b1dfcec4b64db470ed7cc5f2bb265b9b6b5. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Parameter 'function'=<function preprocess_function at 0x000001A8041DEF80> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/1 [00:00<?, ?ba/s]

KeyError: 'question'